In [2]:
__author__ = 'Alexander Soroka, soroka.a.m@gmail.com'
__copyright__ = """Copyright 2020 Alexander Soroka"""



import argparse
import glob
import numpy as np
import tensorflow as tf
import time
from tensorflow.python import keras as keras
from tensorflow.python.keras.callbacks import LearningRateScheduler
from tensorflow.python.keras.callbacks import ModelCheckpoint


LOG_DIR = 'logs'
SHUFFLE_BUFFER = 4
BATCH_SIZE = 64
NUM_CLASSES = 6
PARALLEL_CALLS=4
RESIZE_TO = 224
TRAINSET_SIZE = 14034
VALSET_SIZE = 3000


def parse_proto_example(proto):
    keys_to_features = {
        'image/encoded': tf.io.FixedLenFeature((), tf.string, default_value=''),
        'image/class/label': tf.io.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64))
    }
    example = tf.io.parse_single_example(proto, keys_to_features)
    example['image'] = tf.image.decode_jpeg(example['image/encoded'], channels=3)
    example['image'] = tf.image.convert_image_dtype(example['image'], dtype=tf.float32)
    example['image'] = tf.image.resize(example['image'], tf.constant([RESIZE_TO, RESIZE_TO]))
    return example['image'], tf.one_hot(example['image/class/label'], depth=NUM_CLASSES)


def normalize(image, label):
    return tf.image.per_image_standardization(image), label

def resize(image, label):
    return tf.image.resize(image, tf.constant([RESIZE_TO, RESIZE_TO])), label

def create_dataset(filenames, batch_size):
    """Create dataset from tfrecords file
    :tfrecords_files: Mask to collect tfrecords file of dataset
    :returns: tf.data.Dataset
    """
    return tf.data.TFRecordDataset(filenames)\
        .map(parse_proto_example)\
        .map(resize)\
        .map(normalize)\
        .batch(batch_size)\
        .prefetch(batch_size)


def build_model():
    base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
    )
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()     
    prediction_layer = tf.keras.layers.Dense(NUM_CLASSES, activation=tf.keras.activations.softmax)

    base_model.trainable = False

    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Flatten()(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    
    model.load_weights('SMOMI/weights.hdf5')
    model.trainable = True

    return model



def main():

    train_path = 'SMOMI/dataset/train*'
    test_path = 'SMOMI/dataset/val*'

    train_dataset = create_dataset(glob.glob(train_path), BATCH_SIZE)
    validation_dataset = create_dataset(glob.glob(test_path), BATCH_SIZE)

    
    model = build_model()

    model.compile(
        optimizer=tf.optimizers.SGD(lr=4.2e-10, momentum=0.9),
        loss=tf.keras.losses.categorical_crossentropy,
        metrics=[tf.keras.metrics.categorical_accuracy],
    )

    log_dir='SMOMI/{}/lab3_4-3/ilcd-{}'.format(LOG_DIR, time.time())
    
    
    weights_file = "SMOMI/weightsupd.hdf5"
    
    with tf.device("/gpu:0"):
        model.fit(
        train_dataset,
        epochs=200,
        validation_data=validation_dataset,
        callbacks=[
            tf.keras.callbacks.TensorBoard(log_dir),
            tf.keras.callbacks.ModelCheckpoint(filepath=weights_file, monitor='val_categorical_accuracy', mode='max', save_best_only=True, save_weights_only=True, verbose=1)
            ]
        )
    #model.save_weights('/content/drive/My Drive/SMOMI/weights.h5')

if __name__ == '__main__':
    main()

Epoch 1/200
      1/Unknown - 0s 0s/step - loss: 0.4528 - categorical_accuracy: 0.9062WARNING:tensorflow:From f:\python-anaconda\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
    220/Unknown - 226s 1s/step - loss: 0.4057 - categorical_accuracy: 0.7064
Epoch 00001: val_categorical_accuracy improved from -inf to 0.73000, saving model to SMOMI\weightsupd.hdf5
220/220 [==============================] - 278s 1s/step - loss: 0.4057 - categorical_accuracy: 0.7064 - val_loss: 0.3618 - val_categorical_accuracy: 0.7300
Epoch 2/200
220/220 [==============================] - ETA: 0s - loss: 0.4065 - categorical_accuracy: 0.7054
Epoch 00002: val_categorical_accuracy improved from 0.73000 to 0.73133, saving model to SMOMI\weightsupd.hdf5
220/220 [==============================] - 265s 1s/step - l

Epoch 23/200
220/220 [==============================] - ETA: 0s - loss: 0.3899 - categorical_accuracy: 0.7083
Epoch 00023: val_categorical_accuracy improved from 0.73733 to 0.73767, saving model to SMOMI\weightsupd.hdf5
220/220 [==============================] - 264s 1s/step - loss: 0.3899 - categorical_accuracy: 0.7083 - val_loss: 0.3446 - val_categorical_accuracy: 0.7377
Epoch 24/200
220/220 [==============================] - ETA: 0s - loss: 0.3882 - categorical_accuracy: 0.7106
Epoch 00024: val_categorical_accuracy did not improve from 0.73767
220/220 [==============================] - 265s 1s/step - loss: 0.3882 - categorical_accuracy: 0.7106 - val_loss: 0.3438 - val_categorical_accuracy: 0.7377
Epoch 25/200
220/220 [==============================] - ETA: 0s - loss: 0.3839 - categorical_accuracy: 0.7118
Epoch 00025: val_categorical_accuracy did not improve from 0.73767
220/220 [==============================] - 264s 1s/step - loss: 0.3839 - categorical_accuracy: 0.7118 - val_loss: 

220/220 [==============================] - ETA: 0s - loss: 0.3689 - categorical_accuracy: 0.7171
Epoch 00046: val_categorical_accuracy did not improve from 0.74433
220/220 [==============================] - 256s 1s/step - loss: 0.3689 - categorical_accuracy: 0.7171 - val_loss: 0.3301 - val_categorical_accuracy: 0.7443
Epoch 47/200
220/220 [==============================] - ETA: 0s - loss: 0.3670 - categorical_accuracy: 0.7190
Epoch 00047: val_categorical_accuracy improved from 0.74433 to 0.74467, saving model to SMOMI\weightsupd.hdf5
220/220 [==============================] - 256s 1s/step - loss: 0.3670 - categorical_accuracy: 0.7190 - val_loss: 0.3296 - val_categorical_accuracy: 0.7447
Epoch 48/200
220/220 [==============================] - ETA: 0s - loss: 0.3679 - categorical_accuracy: 0.7181
Epoch 00048: val_categorical_accuracy improved from 0.74467 to 0.74533, saving model to SMOMI\weightsupd.hdf5
220/220 [==============================] - 256s 1s/step - loss: 0.3679 - categorical

Epoch 70/200
220/220 [==============================] - ETA: 0s - loss: 0.3562 - categorical_accuracy: 0.7237
Epoch 00070: val_categorical_accuracy did not improve from 0.74833
220/220 [==============================] - 257s 1s/step - loss: 0.3562 - categorical_accuracy: 0.7237 - val_loss: 0.3183 - val_categorical_accuracy: 0.7480
Epoch 71/200
220/220 [==============================] - ETA: 0s - loss: 0.3548 - categorical_accuracy: 0.7277
Epoch 00071: val_categorical_accuracy did not improve from 0.74833
220/220 [==============================] - 257s 1s/step - loss: 0.3548 - categorical_accuracy: 0.7277 - val_loss: 0.3178 - val_categorical_accuracy: 0.7483
Epoch 72/200
220/220 [==============================] - ETA: 0s - loss: 0.3558 - categorical_accuracy: 0.7207
Epoch 00072: val_categorical_accuracy did not improve from 0.74833
220/220 [==============================] - 257s 1s/step - loss: 0.3558 - categorical_accuracy: 0.7207 - val_loss: 0.3175 - val_categorical_accuracy: 0.7483
E

Epoch 94/200
220/220 [==============================] - ETA: 0s - loss: 0.3464 - categorical_accuracy: 0.7300
Epoch 00094: val_categorical_accuracy did not improve from 0.75200
220/220 [==============================] - 265s 1s/step - loss: 0.3464 - categorical_accuracy: 0.7300 - val_loss: 0.3090 - val_categorical_accuracy: 0.7520
Epoch 95/200
220/220 [==============================] - ETA: 0s - loss: 0.3458 - categorical_accuracy: 0.7274
Epoch 00095: val_categorical_accuracy improved from 0.75200 to 0.75233, saving model to SMOMI\weightsupd.hdf5
220/220 [==============================] - 266s 1s/step - loss: 0.3458 - categorical_accuracy: 0.7274 - val_loss: 0.3087 - val_categorical_accuracy: 0.7523
Epoch 96/200
220/220 [==============================] - ETA: 0s - loss: 0.3426 - categorical_accuracy: 0.7296
Epoch 00096: val_categorical_accuracy did not improve from 0.75233
220/220 [==============================] - 268s 1s/step - loss: 0.3426 - categorical_accuracy: 0.7296 - val_loss: 

Epoch 118/200
220/220 [==============================] - ETA: 0s - loss: 0.3362 - categorical_accuracy: 0.7309
Epoch 00118: val_categorical_accuracy did not improve from 0.75533
220/220 [==============================] - 268s 1s/step - loss: 0.3362 - categorical_accuracy: 0.7309 - val_loss: 0.3012 - val_categorical_accuracy: 0.7547
Epoch 119/200
220/220 [==============================] - ETA: 0s - loss: 0.3348 - categorical_accuracy: 0.7313
Epoch 00119: val_categorical_accuracy did not improve from 0.75533
220/220 [==============================] - 257s 1s/step - loss: 0.3348 - categorical_accuracy: 0.7313 - val_loss: 0.3011 - val_categorical_accuracy: 0.7547
Epoch 120/200
220/220 [==============================] - ETA: 0s - loss: 0.3345 - categorical_accuracy: 0.7324
Epoch 00120: val_categorical_accuracy did not improve from 0.75533
220/220 [==============================] - 257s 1s/step - loss: 0.3345 - categorical_accuracy: 0.7324 - val_loss: 0.3007 - val_categorical_accuracy: 0.754

220/220 [==============================] - ETA: 0s - loss: 0.3281 - categorical_accuracy: 0.7334
Epoch 00142: val_categorical_accuracy did not improve from 0.75667
220/220 [==============================] - 255s 1s/step - loss: 0.3281 - categorical_accuracy: 0.7334 - val_loss: 0.2948 - val_categorical_accuracy: 0.7567
Epoch 143/200
220/220 [==============================] - ETA: 0s - loss: 0.3287 - categorical_accuracy: 0.7351
Epoch 00143: val_categorical_accuracy did not improve from 0.75667
220/220 [==============================] - 255s 1s/step - loss: 0.3287 - categorical_accuracy: 0.7351 - val_loss: 0.2945 - val_categorical_accuracy: 0.7567
Epoch 144/200
220/220 [==============================] - ETA: 0s - loss: 0.3270 - categorical_accuracy: 0.7343
Epoch 00144: val_categorical_accuracy improved from 0.75667 to 0.75700, saving model to SMOMI\weightsupd.hdf5
220/220 [==============================] - 256s 1s/step - loss: 0.3270 - categorical_accuracy: 0.7343 - val_loss: 0.2943 - va

220/220 [==============================] - ETA: 0s - loss: 0.3225 - categorical_accuracy: 0.7352
Epoch 00166: val_categorical_accuracy did not improve from 0.75767
220/220 [==============================] - 268s 1s/step - loss: 0.3225 - categorical_accuracy: 0.7352 - val_loss: 0.2894 - val_categorical_accuracy: 0.7570
Epoch 167/200
220/220 [==============================] - ETA: 0s - loss: 0.3210 - categorical_accuracy: 0.7377
Epoch 00167: val_categorical_accuracy did not improve from 0.75767
220/220 [==============================] - 260s 1s/step - loss: 0.3210 - categorical_accuracy: 0.7377 - val_loss: 0.2891 - val_categorical_accuracy: 0.7573
Epoch 168/200
220/220 [==============================] - ETA: 0s - loss: 0.3177 - categorical_accuracy: 0.7392
Epoch 00168: val_categorical_accuracy did not improve from 0.75767
220/220 [==============================] - 263s 1s/step - loss: 0.3177 - categorical_accuracy: 0.7392 - val_loss: 0.2888 - val_categorical_accuracy: 0.7577
Epoch 169/20

220/220 [==============================] - ETA: 0s - loss: 0.3159 - categorical_accuracy: 0.7413
Epoch 00190: val_categorical_accuracy improved from 0.75833 to 0.75867, saving model to SMOMI\weightsupd.hdf5
220/220 [==============================] - 255s 1s/step - loss: 0.3159 - categorical_accuracy: 0.7413 - val_loss: 0.2848 - val_categorical_accuracy: 0.7587
Epoch 191/200
220/220 [==============================] - ETA: 0s - loss: 0.3165 - categorical_accuracy: 0.7371
Epoch 00191: val_categorical_accuracy did not improve from 0.75867
220/220 [==============================] - 255s 1s/step - loss: 0.3165 - categorical_accuracy: 0.7371 - val_loss: 0.2846 - val_categorical_accuracy: 0.7580
Epoch 192/200
220/220 [==============================] - ETA: 0s - loss: 0.3175 - categorical_accuracy: 0.7378
Epoch 00192: val_categorical_accuracy did not improve from 0.75867
220/220 [==============================] - 255s 1s/step - loss: 0.3175 - categorical_accuracy: 0.7378 - val_loss: 0.2843 - va

In [5]:
%load_ext tensorboard
%tensorboard --logdir "SMOMI/logs"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 11104), started 1 day, 15:14:37 ago. (Use '!kill 11104' to kill it.)